# 분석 일자
(2021.05.05)

## 크롤링 구조

- 드라이버 객체 생성
- 드라이버 객체를 통한 렌더링 (크레딧잡 접속)
    - for문 - 10개의 기업
    - 특정 기업 ID를 검색어로 지정
    - 특정 기업 검색어 엔터
        - for문 - 15년부터 19년까지
        - 총 인원수 크롤링
        - 날짜 크롤링
        - 평균급여 크롤링

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys # 엔터키 입력용
import pandas as pd
import time
# from webdriver_manager.chrome import ChromeDeriverManager

In [2]:
# 코드

# 드라이버
driver = webdriver.Chrome('C:/Temp/chromedriver') # 웹드라이버 객체 생성

# 구글을 열어서 키워드 입력 후 이동
driver.get('https://kreditjob.com/') # 구글 홈페이지 내용 렌더링
time.sleep(1)


# 미리 합칠 데이터프레임을 만들어 봅시다.
all_df = pd.DataFrame(columns=['회사ID', '월별_직원수','년월','월별_평균급여'])


# 크롤링을 해봅시다
# 기업 리스트
unicorn = ['우아한형제들',  '크래프톤', '야놀자', '블랭크코퍼레이션', '뱅크웨어글로벌', '비바리퍼블리카'] 

for name in unicorn :
    
    print(name,"기업 데이터 크롤링을 진행하고 있습니다.")
    
    
    driver.get('https://kreditjob.com/') # 다시 열어줘야만 오류가 안난다...?
    time.sleep(1)
    
    # 검색
    target=driver.find_element_by_css_selector("[class = 'search-query']") # 검색창 설정
    target.send_keys(name) # 검색어 지정
    target.send_keys(Keys.ENTER) # 엔터처리
    time.sleep(4) # 홈페이지가 구성될 때까지 좀 기다려볼까요?


    # 수집할 데이터 컬럼 설정
    name_num = [] # 회사ID
    worker_num = [] # 월별_직원수
    date_num = [] # 년도
    salary_num = [] # 월별_평균급여

    for x in range(2,52):

        # 회사ID는 행마다 넣어줍시다.
        name_num.append(name)

        # 월별_직원수 클릭해서 보기 ==> 5부터 시작 / 55까지 해보자
        data = driver.find_element_by_css_selector('#lineGraph > g:nth-child('+str(x+3)+')')
        data.click()

        # 월별_직원수 가져오기
        try :
            worker_address = '#lineGraph > g.graph-popup'
            worker = driver.find_element_by_css_selector(worker_address)
            worker_text = worker.text
            check = worker_text.split(sep="\n")[1]
            worker_num.append(check)
        except :
            worker_num.append(0)


        # 클릭해서 닫기
        quit = driver.find_element_by_css_selector('#lineGraph > g.graph-popup > g > polygon:nth-child(2)')
        quit.click()

        # 날짜 끌어오기 ==> 2부터 시작 52까지 해보자
        try :
            date_address = '#groupBarGraph > g.axis.xAxis > g:nth-child('+str(x)+')'
            date = driver.find_element_by_css_selector(date_address)
            date_num.append(date.text)
        except :
            date_num.append(0)

    
    # 평균급여 페이지로 넘어가기
    salary_mean_page = driver.find_element_by_css_selector('#root > div > div.body-container > div.company-container > div.company-wrapper > div.company-contents > section.company-monthly-container > div:nth-child(2) > div > div > button.btn.btn.-default')
    salary_mean_page.click()
    
    time.sleep(2) # 렌더링할 시간은 드려야죠.
    
    # for문으로 19년도까지의 데이터를 불러옵니다.
    for y in range(6,56):

        # 평균급여 클릭해보자
        salary_popup = driver.find_element_by_css_selector('#salaryGraph > g:nth-child('+str(y)+')')
        salary_popup.click()

        # 평균급여를 가져와보자
        try :
            salary_address = '#salaryGraph > g.graph-popup'
            salary = driver.find_element_by_css_selector(salary_address)
            salary_total_text = salary.text
            salary_text = salary_total_text.split(sep='\n')[-1]
            salary_num.append(salary_text)
        except :
            salary_num.append(0)

        # 클릭해서 닫기
        quit_again = driver.find_element_by_css_selector('#salaryGraph > g.graph-popup > g > polygon:nth-child(2)')
        quit_again.click()    

    new_df = pd.DataFrame([name_num, worker_num, date_num, salary_num]) # 행 단위로 합쳐진다.
    new_df = new_df.T # 행을 열로 바꾼다
    new_df.columns = ['회사ID', '월별_직원수','년월','월별_평균급여'] # 각 열의 이름을 통합할 데이터프레임의 열과 동일하게 맞춘다.
    all_df = pd.concat([all_df, new_df]) # 각 기업의 데이터를 통합한다.


    driver.back() # 검색창으로 뒤로가기

print("크롤링이 종료되었습니다.")
print()

display(all_df)

우아한형제들 기업 데이터 크롤링을 진행하고 있습니다.
크래프톤 기업 데이터 크롤링을 진행하고 있습니다.
야놀자 기업 데이터 크롤링을 진행하고 있습니다.
블랭크코퍼레이션 기업 데이터 크롤링을 진행하고 있습니다.
뱅크웨어글로벌 기업 데이터 크롤링을 진행하고 있습니다.
비바리퍼블리카 기업 데이터 크롤링을 진행하고 있습니다.
크롤링이 종료되었습니다.



,회사ID,월별_직원수,년월,월별_평균급여
0,우아한형제들,216 명,'15 11월,"2,993,545 원"
1,우아한형제들,221 명,'15 12월,"2,974,596 원"
2,우아한형제들,219 명,'16 1월,"3,097,467 원"
3,우아한형제들,222 명,'16 2월,"2,994,015 원"
4,우아한형제들,226 명,'16 3월,"3,055,818 원"
...,...,...,...,...
45,비바리퍼블리카,264 명,'19 8월,"4,200,937 원"
46,비바리퍼블리카,272 명,'19 9월,"4,250,067 원"
47,비바리퍼블리카,298 명,'19 10월,"4,266,147 원"
48,비바리퍼블리카,310 명,'19 11월,"4,187,554 원"


In [3]:
all_df.to_csv("./크롤링데이터.csv")

In [4]:
check_df = pd.read_csv("./크롤링데이터.csv")
check_df.drop(["Unnamed: 0"], axis=1, inplace=True)
import re
def cleanText(readData):
    #텍스트에 포함되어 있는 문자 제거
    number = re.sub('[^0-9]', '', readData)
    return number

In [5]:
# 한글 없애기
for x in range(len(check_df)):
    check_df.iloc[x][1] = cleanText(check_df.iloc[x][1])
    check_df.iloc[x][2] = cleanText(check_df.iloc[x][2])
    check_df.iloc[x][3] = cleanText(check_df.iloc[x][3])
    
    # 만약 1~9월 사이의 데이터라면?
    if len(check_df.iloc[x][2]) == 3 :
        check_df.iloc[x][2] = check_df.iloc[x][2][:2] + str(0) + check_df.iloc[x][2][2:]

In [6]:
display(check_df)

,회사ID,월별_직원수,년월,월별_평균급여
0,우아한형제들,216,1511,2993545
1,우아한형제들,221,1512,2974596
2,우아한형제들,219,1601,3097467
3,우아한형제들,222,1602,2994015
4,우아한형제들,226,1603,3055818
...,...,...,...,...
295,비바리퍼블리카,264,1908,4200937
296,비바리퍼블리카,272,1909,4250067
297,비바리퍼블리카,298,1910,4266147
298,비바리퍼블리카,310,1911,4187554


In [7]:
check_df.to_excel("./파이널.xlsx")